In [1]:
import sys
import os
from subprocess import check_output
from skimage.util import pad
from skimage.io import imread, imsave
import numpy as np
import cPickle as pickle
import cv2
import time

from joblib import Parallel, delayed
from utilities2014 import execute_command

In [2]:
stack = 'MD585'
first_sec = 48
last_sec = 180

# stack = 'MD579'
# first_sec = 59
# last_sec = 188

bg_color = (230,232,235)

In [ ]:
input_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_renamed'
filenames = os.listdir(input_dir)
suffix = 'thumbnail'
ext = 'tif'
all_files = dict(sorted([(int(img_fn[:-4].split('_')[1]), img_fn) 
                         for img_fn in filenames if img_fn.endswith('_' + suffix + '.' + ext)]))

# the resolution of lossless is "scale factor" times the resolution on which transform parameters are obtained
if suffix == 'lossless':
    scale_factor = 32
elif suffix == 'lossy':
    scale_factor = 16
elif suffix == 'thumbnail':
    scale_factor = 1
    
    
def identify_shape(img_fn):
    return map(int, check_output("identify -format %%Wx%%H %s" % os.path.join(input_dir, img_fn), 
                                 shell=True).split('x'))

img_shapes_arr = np.array(Parallel(n_jobs=16)(delayed(identify_shape)(img_fn) for img_fn in all_files.itervalues()))
max_width = img_shapes_arr[:,0].max()
max_height = img_shapes_arr[:,1].max()

margin = 0

canvas_width = max_width + 2 * margin
canvas_height = max_height + 2 * margin

padded_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_' + suffix + '_padded'
if not os.path.exists(padded_dir):
    os.makedirs(padded_dir)

warped_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_' + suffix + '_warped'
if not os.path.exists(warped_dir):
    os.makedirs(warped_dir)
    
cropped_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_' + suffix + '_cropped'
if not os.path.exists(cropped_dir):
    os.makedirs(cropped_dir)
    


In [3]:
input_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_renamed'
filenames = os.listdir(input_dir)
# suffix = 'thumbnail'
suffix = 'lossless'
ext = 'tif'
all_files = dict(sorted([(int(img_fn[:-4].split('_')[1]), img_fn) 
                         for img_fn in filenames if img_fn.endswith('_' + suffix + '.' + ext)]))

# the resolution of lossless is "scale factor" times the resolution on which transform parameters are obtained
if suffix == 'lossless':
    scale_factor = 32
elif suffix == 'lossy':
    scale_factor = 16
elif suffix == 'thumbnail':
    scale_factor = 1

In [4]:
# img_shapes = {}
# for i, img_fn in all_files.iteritems():
#     img_shapes[i] = np.array(check_output("identify %s" % os.path.join(input_dir, img_fn), shell=True).split()[2].split('x'),
#                      np.int)
    
def identify_shape(img_fn):
    return map(int, check_output("identify -format %%Wx%%H %s" % os.path.join(input_dir, img_fn), 
                                 shell=True).split('x'))

img_shapes_arr = np.array(Parallel(n_jobs=16)(delayed(identify_shape)(img_fn) for img_fn in all_files.itervalues()))
max_width = img_shapes_arr[:,0].max()
max_height = img_shapes_arr[:,1].max()

margin = 0

canvas_width = max_width + 2 * margin
canvas_height = max_height + 2 * margin

In [5]:
padded_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_' + suffix + '_padded'
if not os.path.exists(padded_dir):
    os.makedirs(padded_dir)

warped_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_' + suffix + '_warped'
if not os.path.exists(warped_dir):
    os.makedirs(warped_dir)
    
cropped_dir = '/home/yuncong/csd395/CSHL_data/' + stack + '_' + suffix + '_cropped'
if not os.path.exists(cropped_dir):
    os.makedirs(cropped_dir)

In [6]:
def pad_IM(img_fn):
    
    img_fn = os.path.basename(img_fn)
        
    warped_fn = img_fn[:-4] + '_padded_warped.' + ext
    padded_fn = img_fn[:-4] + '_padded.' + ext
    
    d = {'input_fn': os.path.join(input_dir, img_fn),
        'output_fn': os.path.join(padded_dir, padded_fn),
         'bg_r': bg_color[0],
         'bg_g': bg_color[1],
         'bg_b': bg_color[2],
         'width': canvas_width,
         'height': canvas_height,
         }
        
    convert_cmd = 'convert %(input_fn)s -background "rgb(%(bg_r)d,%(bg_g)d,%(bg_b)d)" -gravity center -extent %(width)dx%(height)d -compress lzw %(output_fn)s'%d
    execute_command(convert_cmd)
    
def warp_IM(img_fn, T):
    
    warped_fn = img_fn[:-4] + '_padded_warped.' + ext
    padded_fn = img_fn[:-4] + '_padded.' + ext

    d = {'sx':T[0,0],
         'sy':T[1,1],
         'rx':T[1,0],
         'ry':T[0,1],
         'tx':T[0,2],
         'ty':T[1,2],
         'input_fn': os.path.join(padded_dir, padded_fn),
         'output_fn': os.path.join(warped_dir, warped_fn)
        }

    affine_cmd = "convert %(input_fn)s -distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' %(output_fn)s"%d
    execute_command(affine_cmd)
    
def crop_IM(img_fn, x, y, w, h):
    
    warped_fn = img_fn[:-4] + '_padded_warped.' + ext
    cropped_fn = img_fn[:-4] + '_padded_warped_cropped.' + ext

    d = {
    'x': x * scale_factor,
    'y': y * scale_factor,
    'w': w * scale_factor,
    'h': h * scale_factor,
    'input_fn': os.path.join(warped_dir, warped_fn),
    'output_fn': os.path.join(cropped_dir, cropped_fn)
    }

    crop_cmd = 'convert %(input_fn)s -crop %(w)dx%(h)d+%(x)d+%(y)d %(output_fn)s'%d
    execute_command(crop_cmd)

In [ ]:
t = time.time()

# Parallel(n_jobs=2)(pad_IM(all_files[secind]) for secind in range(first_sec, last_sec+1))

# Parallel(n_jobs=3)(pad_IM(all_files[secind]) for secind in range(first_sec, first_sec+2))

for secind in range(first_sec, last_sec+1):
    pad_IM(all_files[secind])

# pad_IM(all_files[first_sec+1])

print time.time() - t

convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0048_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0048_lossless_padded.tif
convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0049_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0049_lossless_padded.tif

Child returned 0
Child returned


convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0050_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0050_lossless_padded.tif

 0
Child returned


convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0051_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0051_lossless_padded.tif

...


convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0060_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0060_lossless_padded.tif

 0
Child returned


convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0061_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0061_lossless_padded.tif

 0
Child returned


convert /home/yuncong/csd395/CSHL_data/MD585_renamed/MD585_0062_lossless.tif -background "rgb(230,232,235)" -gravity center -extent 36324x27066 -compress lzw /home/yuncong/csd395/CSHL_data/MD585_lossless_padded/MD585_0062_lossless_padded.tif

In [9]:
if not os.path.exists('/home/yuncong/csd395/CSHL_data/%s_thumbnail_finalTransfParams.pkl'%stack):
    execute_command('../elastix/successive_registration.py %s %d %d thumbnail' % (stack, first_sec, last_sec))

In [15]:
with open('/home/yuncong/csd395/CSHL_data/%s_thumbnail_finalTransfParams.pkl'%stack, 'r') as f:
    Ts = pickle.load(f)
    
Ts_lossless = {}
for sec, T in Ts.iteritems():
    T_lossless = T.copy()
    T_lossless[:2, 2] = T[:2, 2] * scale_factor
    Ts_lossless[sec] = T_lossless

In [ ]:
t = time.time()

for secind in range(first_sec, last_sec+1):
    warp_IM(all_files[secind], np.linalg.inv(Ts_lossless[secind]))

# warp_IM(all_files[first_sec], np.linalg.inv(Ts_lossless[first_sec]))

print time.time() - t

In [ ]:
    
t = time.time()

for secind in range(first_sec, last_sec+1):
#     crop_IM(all_files[secind],576,413,403,280) #MD579
    crop_IM(all_files[secind],543,440,489,248) #MD585

# crop_IM(all_files[first_sec],543,440,489,248) #MD585

print time.time() - t

In [18]:
# def pad_and_warp_and_crop(img_fn, T, input_dir=input_dir, padded_dir=padded_dir, 
#                                       warped_dir=warped_dir, cropped_dir=cropped_dir,
#                           ext='tif'):
    
#     img_fn = os.path.basename(img_fn)    
#     secind = int(img_fn[:-4].split('_')[1])    
    
#     warped_fn = img_fn[:-4] + '_padded_warped.' + ext
#     padded_fn = img_fn[:-4] + '_padded.' + ext
    
#     d = {'input_fn': os.path.join(input_dir, img_fn),
#         'output_fn': os.path.join(padded_dir, padded_fn),
#          'bg_r': bg_color[0],
#          'bg_g': bg_color[1],
#          'bg_b': bg_color[2],
#          'width': canvas_width,
#          'height': canvas_height,
#          }
        
#     convert_cmd = 'convert %(input_fn)s -background "rgb(%(bg_r)d,%(bg_g)d,%(bg_b)d)" -gravity center -extent %(width)dx%(height)d -compress lzw %(output_fn)s'%d
#     execute_command(convert_cmd)
    
#     # special treatment for MD579
#     if secind in range(59, 67):
#         d = {
#             'input_fn': os.path.join(padded_dir, padded_fn),
#             'output_fn': os.path.join(padded_dir, padded_fn),
#             'bg_r': bg_color[0],
#             'bg_g': bg_color[1],
#             'bg_b': bg_color[2],
#             'offset_y': '-' + str(300*scale_factor)    
#         }
#         execute_command('convert -page +0%(offset_y)s %(input_fn)s -background "rgb(%(bg_r)d,%(bg_g)d,%(bg_b)d)" -flatten %(output_fn)s'%d)

#     d = {'sx':T[0,0],
#          'sy':T[1,1],
#          'rx':T[1,0],
#          'ry':T[0,1],
#          'tx':T[0,2],
#          'ty':T[1,2],
#          'input_fn': os.path.join(padded_dir, padded_fn),
#          'output_fn': os.path.join(warped_dir, warped_fn)
#         }

#     affine_cmd = "convert %(input_fn)s -distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' %(output_fn)s"%d
#     execute_command(affine_cmd)
    
#     cropped_fn = img_fn[:-4] + '_padded_warped_cropped.' + ext

#     d = {
#     'x': 576 * scale_factor,
#     'y': 413 * scale_factor,
#     'w': 403 * scale_factor,
#     'h': 280 * scale_factor,
#     'input_fn': os.path.join(warped_dir, warped_fn),
#     'output_fn': os.path.join(cropped_dir, cropped_fn)
#     }

#     crop_cmd = 'convert %(input_fn)s -crop %(w)dx%(h)d+%(x)d+%(y)d %(output_fn)s'%d
#     execute_command(crop_cmd)
    
#     d = {
#         'input_fn': os.path.join(warped_dir, warped_fn),
#         'shrinked_fn': os.path.join(shrinked_dir, img_fn[:-4] + '_padded_warped_shrinked.' + ext)
#     }
        
#     shrink_cmd = 'convert %(input_fn)s -scale 12.5%% %(shrinked_fn)s'%d
#     execute_command(shrink_cmd)

In [ ]:
# for secind in range(first_sec, last_sec+1):
#     t = time.time()
#     pad_and_warp_and_crop(all_files[secind], np.linalg.inv(Ts_lossless[secind]))
#     print time.time()-t

In [9]:
# t = time.time()
# #does not save much time
# Parallel(n_jobs=3)(delayed(pad_and_warp_and_crop)(all_files[secind], np.linalg.inv(Ts_lossless[secind])) 
#                                           for secind in Ts_lossless.keys()[4:7])
# print time.time() - t